In [5]:
#抓取百度数据
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np
import pandas as pd
import re
import os

from datetime import datetime,timedelta
thedate=datetime.strftime(datetime.today()-timedelta(days=1),"%Y-%m-%d")
thedate2=datetime.strftime(datetime.today()-timedelta(days=1),"%m.%d")

browser = webdriver.Chrome()
url='https://motion.baidu.com/activity/yiqing_predict/main?province=%E5%85%A8%E5%9B%BD'
browser.get(url)

with open(r'./COVIDend/%s.html' % thedate,'w',encoding='utf-8') as f:
    f.write(browser.page_source)

browser.close()

In [6]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])
city_popu.head(2)


from bs4 import BeautifulSoup
import lxml
# thedate="2022-12-26"
soup=BeautifulSoup(open(r'COVIDend\%s.html'  % thedate,'r',encoding='utf-8').read())

trs=soup.find_all('div',{"class":'list-item_1JNax'})

def get_result(tr):
    city=tr.find('div',{'class':'list-item-city_qLfui pc_UIzzf'}).find('div').text.replace('伊犁哈萨克自治州','伊犁')
    progress=tr.find('div',{'class':'rate-text_1PWuo'}).text
    peak=tr.find_all('div',{'class':'flex_3nTxm'})[1].text
    end=tr.find('div',{'class':'flex_3nTxm end_MnHhu'}).text
    return [city.replace('市',''),progress,peak,end]

d=[get_result(tr) for tr in trs]

df=pd.DataFrame(data=d,columns=['city','progress','peak','end'])
df=df.sort_values(by='end')
df=df.merge(city_popu,left_on='city',right_on='城市')

def covid_progress(s):
    s=s.replace('%','')
    if '回落至' in s:
        s=s.replace('回落至','')
    if '上升至' in s:
        s=s.replace('上升至','')
        s=200-int(s)
    return int(s)    
df['progress_2']=df['progress'].apply(covid_progress)
print(thedate)

2022-12-28


In [7]:
df.to_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
print(thedate)

2022-12-28


In [13]:
# title=pd.read_clipboard()
title.merge(df[['city','progress']],how='left',on='city').to_clipboard(index=False)

In [9]:
df.to_clipboard()

In [4]:
#COVID END date

allpeople=df['常住人口'].sum()
# allpeople
df2=df.groupby('end').sum().reset_index()
df2['人口cumsum']=df2['常住人口'].cumsum()
df2['人口cumsum_per']=df2['人口cumsum']/allpeople
df2['sum']=np.nan
df2[['end','sum','人口cumsum_per']].to_clipboard(index=False,header=None)

In [29]:
#covid progress

def covid_status(s):
    status=""
    if '上升至' in s:
        s=s.replace('上升至','').replace('%','')
        if int(s) <=50:
            status='1上升初期'
        elif 50< int(s) <=80:
            status='2上升中期'
        elif int(s)>80:
            status='3上升后期'
    if '回落至' in s:
        s=s.replace('回落至','').replace('%','')
        if int(s) <=50:
            status='6下降后期'
        elif 50< int(s) <=80:
            status='5下降中期'
        elif int(s)>80:
            status='4下降初期'
    return status

df['status']=df['progress'].apply(covid_status)
df['date']=thedate
df


pv_value=pd.pivot_table(df, values='常住人口', index=['date'],columns=["status"], aggfunc=np.sum)

pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
pv_value['0 sum']=1
# try:
#     pv_value=pv_value[['sum','1上升初期', '2上升中期', '3上升后期', '4下降初期','5下降中期','6下降后期']]
# except:
#     pv_value=pv_value[['sum','1上升初期', '2上升中期', '3上升后期', '4下降初期','5下降中期']]

pv_value=pv_value.fillna(0)
pv_value=pv_value.sort_index(axis=1)
pv_value.tail(22).to_clipboard(header=None)

pv_value
# pv_value.tail(3)

status,0 sum,3上升后期,4下降初期,5下降中期,6下降后期
date,,,,,
2022-12-26,1,0.017552,0.073201,0.28127,0.627977
